# Simulated Annealing on General Assignment Problem

In this project, we tried to solve General Assignment Problem (GAP) with Simulated Annealing (SA).

## General Assignment Problem

There are a number of agents and a number of tasks. Any agent can be assigned to perform any task, incurring some cost. Moreover, each agent has a budget and the sum of the costs of tasks assigned to it cannot exceed this budget. It is required to find an assignment in which all agents do not exceed their budget and total cost of the assignment is minimized.

## Solution Representation

To represent the solution, I used a simple list structure. Cells of the list are jobs and numbers in the cells are the agents assigned to that job.

Example solution representation:
[7, 1, 0, 4, 4, 2, 4, 1, 5, 6, 3, 4, 5, 5, 1, 5, 2, 1, 4, 0, 7, 7, 5, 7, 5, 2, 3, 0, 2, 1, 7, 6, 6, 7, 3, 4, 3, 6, 3, 6]

## Algorithm

We used simulated annealing with some modification to solve General Assignment Problem. First, I will write the pseudocode of my simulated annealing and then try to explain important parts like neighborhood structure and stopping condition.

### Pseudocode

<ol>
<li>Compute an initial solution $X$ and choose an initial temprature $T>0$ and a repetition factor $L$</li>
<li>As long as the stopping criterion is not satisfied perform the following steps:</li>
    <ol>
    <li>Do the following $L_k$ times</li>
        <ol>
        <li>Generate a random solution in the neighborhood $x' \in N(x)$.</li>
        <li>Compute $\Delta = f(x') - f(x) + unfit(x')$ </li>
        <li>Generate a random number $0<u<1$</li>
        <li>If $\Delta<0$ or u < $e^{-\frac{\Delta}{T_k}}$ then set $x \gets x'$ </li>
        <li>If $x$ is feasible and $f(x) < f(x^*)$ then set $x^* \gets x$</li>
        </ol>
    <li>Update $T_k$ and $L_k$</li>
        <ol>
        <li>$T_{k+1} \gets \alpha * T_k$</li>
        <li>$L_{k+1} \gets \gamma * L_k$</li>
        </ol>
    </ol>
<li>Report the incumbent solution</li>
</ol>

### Important Functionalities

In this part, I will explain important parts of the algorith. You can also find the corresponding function's name of the code next to each subsection name.

#### First Solution Generation (generateSolution())

This step generates the first solution for the algorithm. For the reason that SA is an improvement heuristic, we should generate an initial solution. I will explain it with the pseuodo code:

<ol>
<li>While no feasible solution found, loop 1000 times:</li>
    <ol>
    <li>Create a list of jobs and take a random permutation of it then iterate on the list:</li>
        <ol>
        <li>Find the agent with minimum resource consumption on this job and the agent whose resource limit not exceeded</li>
            <ol>
            <li>Assign the job to that user</li>
            </ol>
        <li>If there is no viable/feasible agent for a job, then break</li>
        </ol>
    </ol>
<li>If no feasible solution found, then create a random assignment list.</li>
</ol>



#### Neigborhood Structure (findNeighbor(solution, nSize))

I tried two different neigborhood structure. They are very similar actually.
<ol>
<li>Randomly change assignees of two randomly chosen jobs</li>
<li>Randomly change assignees of three randomly chosen jobs</li>
</ol>

Actually in the code, the nSize parameter of findNeighbor(solution, nSize) function indicates the number of changes of neighbor structure.


#### Unfitness of a Solution (calculateUnfit(solution))

My simulated annealing algorithm can also accept infeasible solutions. However, to punish infeasible solutions, I used an unfitness score which I took from the GA on GAP paper. In short, the unfitness score calculated like this:

$ u_k = \sum_{i \in I} max \left[0, \left(\sum_{j \in J, s_{kj} = i} r_{ij} \right) - b_i \right] $

#### Initial Temperature (setInitialTemp(acceptProb, nSize))

I used the method we learnt in the class. I calculated the first temperature from the initial acceptance probability.

First, I calculated 10000 delta value which are difference between objective values of two neighbor solution. Then took the average of delta values. Then:

$ T_1 = \frac{\left|\Delta_{bar}\right|}{\ln{\frac{1}{P_1}}} $


#### Stopping Condition (stopCond(allSolutions, L))

<ul>
<li>Do not stop while $L < 10000$ </li>
<li>If $L > 10000$ and $L < 50000$ then stop if the change in objective function of $10$ last solution is less than $0.1%$</li>
<li>If $L > 50000$ and $L < 150000$ then stop if the change in objective function of $5$ last solution is less than $0.1%$</li>
<li>If $L > 150000$ stop </li>
</ul>

### Importing important libraries

In [1]:
%matplotlib inline

import numpy as np
import math
import time
import random
import pickle
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

# Enter your own plotly credentials to play on plots
plotly.tools.set_credentials_file(username='', api_key='')

### Reading data
Reading input from document and initializing variables.
You can change dataset variable to 1, 2, 3 or 4 to use different input sets.

In [72]:
# Dataset number. 1, 2, 3 or 4
dataset = "4"

if dataset == "1":
    optimalObjective = 438
elif dataset == "2":
    optimalObjective = 646
elif dataset == "3":
    optimalObjective = 573
else:
    optimalObjective = 1698

filename = "gap-data-4instances" + dataset + ".txt"
f = open(filename, 'r')
l = f.readline().split()

# number of agents
m = int(l[0])

# number of jobs
n = int(l[1])

# cost of assigning job j to agent i
c = []

# resource required by agent i to perform job j
r = []

# resource capacity of agent i.
b = []


for i in range(m):
    temp = []
    for l in f.readline().split():
        temp.append(int(l))
    c.append(temp)
    
for i in range(m):
    temp = []
    for l in f.readline().split():
        temp.append(int(l))
    r.append(temp)

for l in f.readline().split():
    b.append(int(l))
    
f.close()

### Helper Funtions

Functions to be used in Simulated Annealing algorithm.

In [3]:
def isFeasible(solution):
    caps = [0] * m
    for i in range(len(solution)):
        caps[solution[i]] = caps[solution[i]] + r[solution[i]][i]
        
    for i in range(len(caps)):
        if b[i] < caps[i]:
            return False
        
    return True

def calculateCost(solution):
    cost = 0
    for i in range(len(solution)):
        cost = cost + c[solution[i]][i]
    
    return cost

def calculateUnfit(solution):
    unfitness = 0
    caps = [0] * m
    
    for i in range(len(solution)):
        caps[solution[i]] = caps[solution[i]] + r[solution[i]][i]
        
    for i in range(len(caps)):
        unfitness = unfitness + max(0, caps[i] - b[i])
    
    return unfitness

def generateSolution():
    solution = [0] * n
    
    for count in range(1000):
        isFound = True
        caps = [0] * m
        for job in np.random.permutation(n):
            minWork = sum(b)
            minWorker = -1
            for i in range(m):
                if caps[i]+r[i][job] <= b[i]:
                    if r[i][job] < minWork:
                        minWork = r[i][job]
                        minWorker = i
            if minWorker == -1:
                isFound = False
                break
            else:
                solution[job] = minWorker
                caps[minWorker] = caps[minWorker] + minWork
        
        if isFound == True:
            break
    
    if not isFound:
        for i in range(len(solution)):
            solution[i] = random.randint(0,m-1)
    
    return solution


def findNeighbor(solution, nSize):
    newSolution = list(solution)
    
    for i in range(nSize):
        k = random.randint(0,m*n-1)
        agent = int(k / n)
        job = k % n
        newSolution[job] = agent
        
    return newSolution
    
def setInitialTemp(acceptProb, nSize):
    deltaSum = 0
    for i in range(10000):
        solution = generateSolution()
        newSolution = findNeighbor(solution, nSize)
        deltaSum = deltaSum + abs(calculateCost(newSolution) - calculateCost(solution))
    
    return (deltaSum/10000) / math.log(1/acceptProb)


def stopCond(allSolutions, L):
    s = len(allSolutions)
    if L < 10000:
        return False
    if L > 150000:
        return True
    if L > 50000 and ((allSolutions[s-5] - allSolutions[s-1]) / allSolutions[s-5]) * 100 < 0.1:
        return True
    if ((allSolutions[s-10] - allSolutions[s-1]) / allSolutions[s-10]) * 100 > 0.1:
        return False
    
    
    return True
    
    
solution = generateSolution()
print(solution)
print(isFeasible(solution))

newSolution = findNeighbor(solution, 2)
print(newSolution)

print(calculateCost(solution))
print(calculateCost(newSolution))


#print(math.exp(-0.1/0.3))

[7, 1, 0, 4, 4, 2, 4, 1, 5, 6, 3, 4, 5, 5, 1, 5, 2, 1, 4, 0, 7, 7, 5, 7, 5, 2, 3, 0, 2, 1, 7, 6, 6, 7, 3, 4, 3, 6, 3, 6]
True
[7, 1, 0, 4, 4, 2, 4, 1, 0, 6, 3, 4, 5, 5, 1, 5, 2, 5, 4, 0, 7, 7, 5, 7, 5, 2, 3, 0, 2, 1, 7, 6, 6, 7, 3, 4, 3, 6, 3, 6]
819
817


In [4]:
print(setInitialTemp(0.80, 2))

21.802557014741705


### Main Simulated Annealing Code

In [27]:
# nReheat: How many times the algorithm will run with this parameters from the begining.
# bestSolutions: Holds all the incumbent solutions for each run.
# totalIterations: Number of total iterations for each run.

def SA_GAP(initialTemp, initialL, alpha, gama, nSize, nReheat, bestSolutions, totalIterations):
    Temp = initialTemp
    L = initialL
    allSolutions = []
    solution = generateSolution()
    
    bestSolution = []
    if isFeasible(solution):
        bestSolution = list(solution)
    
    reheated = True

    for rh in range(nReheat):
        t11 = time.clock()
        while not stopCond(allSolutions, L):
            if not reheated:
                if bestSolution == []:
                    solution = generateSolution()
                else:
                    solution = list(bestSolution)
            else:
                reheated = False

            for i in range(L):
                newSolution = findNeighbor(solution, nSize)
                u = np.random.uniform(0,1)
                deltaCost = calculateCost(newSolution) - calculateCost(solution) + calculateUnfit(newSolution)

                if deltaCost < 0 or u < math.exp(-deltaCost/(Temp)):
                    solution = list(newSolution)
                if isFeasible(newSolution) and  (bestSolution == [] or calculateCost(newSolution) < calculateCost(bestSolution)):
                    bestSolution = list(newSolution)


            Temp = Temp * alpha
            L = int(L * gama)
            allSolutions.append(calculateCost(bestSolution))
            #print(str(len(allSolutions)) + ' - ' + str(math.exp(-deltaCost/(Temp))) + ' - ' + str(L) + ' - ' + str(calculateCost(bestSolution)) + ' - ' + str(calculateCost(solution)))
            
        t22 = time.clock()
        bestSolutions.append(bestSolution)
        totalIterations.append(len(allSolutions))
        
        bSolVal = calculateCost(bestSolution)
        G = 100 * (bSolVal-optimalObjective) / optimalObjective
        nCycle = len(allSolutions)
        timePassed = t22-t11
        print("Objective Value: " + str(bSolVal) + " - %Gap: " + "%.2f" %G + " - No. of Cycles: " + "%.2f" %nCycle +" - CPU Time(s): " + "%.2f" %timePassed)

        allSolutions = []
        Temp = initialTemp
        L = initialL
        solution = generateSolution()
        bestSolution = []
        if isFeasible(solution):
            bestSolution = list(solution)
        reheated = True


## Hyperparameter Optimization

After coding a working and partially efficient simulated annealing algortihm, the process turns into a hyperparameter optimization problem. Because in simulated annealing, there are lots of parameters you need to decide on and all of them effect can effect both the quality of output and time spent. So, after trying lots of combinations and taking notes on them, I actaully got very confused and decided to make a automated system which will run the code several times with different parameters.

I determined some values for each parameter and run the code 5 times for each adjustment for each dataset. After each run I took objective values, %Gap values, number of cycles and run times for statistics.

We ran the hyperparameter optimization on dataset 2. It actually was going to take more than 8 hours to complete, so to avoid this situation, when we encounter a configuration which gives worse than 2% gap in any run then we discard this configuration. With this method, it took nearly 3 hours to complete.

Because it took so long to complete, I ran the code in two parts, first one for 2-change move operator and second one for 3-change move operator.

** Important Note: It takes too long to run the optimization part. Because of that, I saved the output of the optimization parts into two files. You can load them via the code below the optimization part. **

In [24]:
# Hyperparameter runs for 2-change move

runs = 5

nSizes = [2]

initialProbs = [0.75, 0.85, 0.95]

initialLs = [100, 500, 1000]

alphas = [0.85, 0.90, 0.95]

gamas = [1.05, 1.1, 1.15]

hyperParameters = []

counter = 1

for nSize in nSizes:
    for initialProb in initialProbs:
        for initialL in initialLs:
            for alpha in alphas:
                for gama in gamas:
                    hp = []
                    gapSum = 0
                    isOK = True
                    
                    print("(" + str(counter) + ") nSize: " + str(nSize) + " - Acceptance: " + str(initialProb) + " - L: " + str(initialL) + " - alpha: " + str(alpha) + " - gama: " + str(gama))
                    for run in range(runs):
                        t1 = time.clock()

                        bestSolutions = []
                        totalIterations = []

                        initialTemp = setInitialTemp(initialProb, nSize)
                        
                        SA_GAP(initialTemp, initialL, alpha, gama, nSize, 1, bestSolutions, totalIterations)

                        t2 = time.clock()

                        bSolVal = calculateCost(bestSolutions[0])
                        G = 100 * (bSolVal-optimalObjective) / optimalObjective
                        nCycle = totalIterations[0]
                        timePassed = t2-t1

                        print(str(run) + ". Objective Value: " + str(bSolVal) + " - %Gap: " + "%.2f" %G + " - No. of Cycles: " + "%.2f" %nCycle +" - CPU Time(s): " + "%.2f" %timePassed)
                        
                        hp.append([bSolVal, G, nCycle, timePassed])
                        gapSum = gapSum + G
                        
                        if G > 2:
                            isOK = False
                            break
                        
                    if isOK:
                        hyperParameters.append([hp,counter])
                    else:
                        print("DISCARD")
                    
                    counter = counter + 1
                    print()

(1) nSize: 2 - Acceptance: 0.75 - L: 100 - alpha: 0.85 - gama: 1.05
0. Objective Value: 656 - %Gap: 1.55 - No. of Cycles: 101.00 - CPU Time(s): 11.18
1. Objective Value: 659 - %Gap: 2.01 - No. of Cycles: 97.00 - CPU Time(s): 9.29
DISCARD

(2) nSize: 2 - Acceptance: 0.75 - L: 100 - alpha: 0.85 - gama: 1.1
0. Objective Value: 654 - %Gap: 1.24 - No. of Cycles: 66.00 - CPU Time(s): 19.46
1. Objective Value: 662 - %Gap: 2.48 - No. of Cycles: 57.00 - CPU Time(s): 9.18
DISCARD

(3) nSize: 2 - Acceptance: 0.75 - L: 100 - alpha: 0.85 - gama: 1.15
0. Objective Value: 651 - %Gap: 0.77 - No. of Cycles: 45.00 - CPU Time(s): 13.37
1. Objective Value: 659 - %Gap: 2.01 - No. of Cycles: 39.00 - CPU Time(s): 6.49
DISCARD

(4) nSize: 2 - Acceptance: 0.75 - L: 100 - alpha: 0.9 - gama: 1.05
0. Objective Value: 659 - %Gap: 2.01 - No. of Cycles: 97.00 - CPU Time(s): 8.53
DISCARD

(5) nSize: 2 - Acceptance: 0.75 - L: 100 - alpha: 0.9 - gama: 1.1
0. Objective Value: 657 - %Gap: 1.70 - No. of Cycles: 49.00 - CP

In [25]:
# Writing hyperparameter optimization results to a file

#f = open('dataset2x-2.pckl', 'wb')
#pickle.dump(hyperParameters, f)
#f.close()

In [33]:
# Hyperparameter runs for 3-change move

runs = 5

nSizes = [3]

initialProbs = [0.75, 0.85, 0.95]

initialLs = [100, 500, 1000]

alphas = [0.85, 0.90, 0.95]

gamas = [1.05, 1.1, 1.15]

hyperParameters = []

counter = 1

for nSize in nSizes:
    for initialProb in initialProbs:
        for initialL in initialLs:
            for alpha in alphas:
                for gama in gamas:
                    hp = []
                    gapSum = 0
                    isOK = True
                    
                    print("(" + str(counter) + ") nSize: " + str(nSize) + " - Acceptance: " + str(initialProb) + " - L: " + str(initialL) + " - alpha: " + str(alpha) + " - gama: " + str(gama))
                    for run in range(runs):
                        t1 = time.clock()

                        bestSolutions = []
                        totalIterations = []

                        initialTemp = setInitialTemp(initialProb, nSize)
                        
                        SA_GAP(initialTemp, initialL, alpha, gama, nSize, 1, bestSolutions, totalIterations)

                        t2 = time.clock()

                        bSolVal = calculateCost(bestSolutions[0])
                        G = 100 * (bSolVal-optimalObjective) / optimalObjective
                        nCycle = totalIterations[0]
                        timePassed = t2-t1

                        print(str(run) + ". Objective Value: " + str(bSolVal) + " - %Gap: " + "%.2f" %G + " - No. of Cycles: " + "%.2f" %nCycle +" - CPU Time(s): " + "%.2f" %timePassed)
                        
                        hp.append([bSolVal, G, nCycle, timePassed])
                        gapSum = gapSum + G
                        
                        if G > 2:
                            isOK = False
                            break
                        
                    if isOK:
                        hyperParameters.append([hp,counter])
                    else:
                        print("DISCARD")
                    
                    counter = counter + 1
                    print()

(1) nSize: 3 - Acceptance: 0.75 - L: 100 - alpha: 0.85 - gama: 1.05
0. Objective Value: 655 - %Gap: 1.39 - No. of Cycles: 108.00 - CPU Time(s): 15.51
1. Objective Value: 653 - %Gap: 1.08 - No. of Cycles: 105.00 - CPU Time(s): 13.81
2. Objective Value: 657 - %Gap: 1.70 - No. of Cycles: 104.00 - CPU Time(s): 13.23
3. Objective Value: 658 - %Gap: 1.86 - No. of Cycles: 98.00 - CPU Time(s): 10.94
4. Objective Value: 659 - %Gap: 2.01 - No. of Cycles: 115.00 - CPU Time(s): 22.35
DISCARD

(2) nSize: 3 - Acceptance: 0.75 - L: 100 - alpha: 0.85 - gama: 1.1
0. Objective Value: 658 - %Gap: 1.86 - No. of Cycles: 56.00 - CPU Time(s): 9.91
1. Objective Value: 657 - %Gap: 1.70 - No. of Cycles: 54.00 - CPU Time(s): 8.41
2. Objective Value: 652 - %Gap: 0.93 - No. of Cycles: 64.00 - CPU Time(s): 19.58
3. Objective Value: 666 - %Gap: 3.10 - No. of Cycles: 57.00 - CPU Time(s): 10.78
DISCARD

(3) nSize: 3 - Acceptance: 0.75 - L: 100 - alpha: 0.85 - gama: 1.15
0. Objective Value: 653 - %Gap: 1.08 - No. of Cy

In [34]:
# Writing hyperparameter optimization results to a file

#f = open('dataset2x-3.pckl', 'wb')
#pickle.dump(hyperParameters, f)
#f.close()

### Examining the hyperparameter optimization results

In [6]:
# Loads the optimization results

f = open('dataset2x-2.pckl', 'rb')
hyperParameters2 = pickle.load(f)
f.close()

f = open('dataset2x-3.pckl', 'rb')
hyperParameters3 = pickle.load(f)
f.close()

In [16]:
print("Number of Configurations from 2-Change: " + str(len(hyperParameters2)))
print("Number of Configurations from 3-Change: " + str(len(hyperParameters3)))

print(hyperParameters2[0])
print(hyperParameters3[0])

Number of Configurations from 2-Change: 28
Number of Configurations from 3-Change: 16
[[[654, 1.238390092879257, 45, 13.657973738697706], [650, 0.6191950464396285, 45, 14.015205117781647], [657, 1.7027863777089782, 45, 13.887467985223111], [652, 0.9287925696594427, 48, 19.523433256177896], [649, 0.46439628482972134, 42, 9.196125792245994]], 6]
[[[653, 1.08359133126935, 33, 15.087502908638271], [654, 1.238390092879257, 40, 38.49076919612526], [654, 1.238390092879257, 30, 10.66001620541283], [654, 1.238390092879257, 33, 15.26724867091525], [654, 1.238390092879257, 33, 16.42331841186933]], 12]


In [18]:
hyperParametersAvg2 = []
for i in range(int(len(hyperParameters2))):
    hyperParametersAvg2.append([[0,0,0,0],0])

for i in range(len(hyperParameters2)*5):
    hyperParametersAvg2[int(i/5)][1] = hyperParameters2[int(i/5)][1]
    for j in range(len(hyperParametersAvg2[int(i/5)][0])):
        hyperParametersAvg2[int(i/5)][0][j] = hyperParametersAvg2[int(i/5)][0][j] + hyperParameters2[int(i/5)][0][i%5][j]/5
        
        
hyperParametersAvg3 = []
for i in range(int(len(hyperParameters3))):
    hyperParametersAvg3.append([[0,0,0,0],0])

for i in range(len(hyperParameters3)*5):
    hyperParametersAvg3[int(i/5)][1] = hyperParameters3[int(i/5)][1]
    for j in range(len(hyperParametersAvg3[int(i/5)][0])):
        hyperParametersAvg3[int(i/5)][0][j] = hyperParametersAvg3[int(i/5)][0][j] + hyperParameters3[int(i/5)][0][i%5][j]/5
        
print(hyperParametersAvg2[0])

print(hyperParametersAvg3[0])

[[652.4000000000001, 0.9907120743034055, 45.0, 14.056041178025271], 6]
[[653.8, 1.2074303405572755, 33.800000000000004, 19.18577107859219], 12]


Our main objective is minimizing both Gap percentage and CPU time. So first, I plot the Gap percentage penalties of each run and then plot the CPU times of eah run.

In [21]:

# Create random data with numpy
import numpy as np

x2 = []
x3 = []

hpGaps2 = []
for hp in hyperParametersAvg2:
    hpGaps2.append(hp[0][1])
    x2.append(hp[1])

hpGaps3 = []
for hp in hyperParametersAvg3:
    hpGaps3.append(hp[0][1])
    x3.append(hp[1])


# Create a trace
trace = go.Scatter(
    x = x2,
    y = hpGaps2,
    mode = 'markers',
    name = '2-Changes'
)

trace2 = go.Scatter(
    x = x3,
    y = hpGaps3,
    mode = 'markers',
    name = '3-Changes'
)

data = [trace, trace2]

layout = go.Layout(
    title='Gap Percentage of Runs',
    xaxis=dict(
        title='Run Number',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Gap Percentage',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Gap Percentage of Runs')

#py.iplot(data, filename='basic-scatter')

In [22]:

# Create random data with numpy
import numpy as np

x2 = []
x3 = []

hpTime2 = []
for hp in hyperParametersAvg2:
    hpTime2.append(hp[0][3])
    x2.append(hp[1])

hpTime3 = []
for hp in hyperParametersAvg3:
    hpTime3.append(hp[0][3])
    x3.append(hp[1])


# Create a trace
trace = go.Scatter(
    x = x2,
    y = hpTime2,
    mode = 'markers',
    name = '2-Changes'
)

trace2 = go.Scatter(
    x = x3,
    y = hpTime3,
    mode = 'markers',
    name = '3-Changes'
)

data = [trace, trace2]

layout = go.Layout(
    title='CPU Times of Runs',
    xaxis=dict(
        title='Run Number',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='CPU Times (s)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='CPU Time of Runs')

#py.iplot(data, filename='basic-scatter')

These are all actaully good results, because we have already discarded the parameters which gives gap% worse than %2. However we want the best results, so let's also take parameters with gap% greater than 1. 

However CPU time graph is more scattered and it is hard to observe it. So lets cut runs from 15 seconds. We will plot runs that executed in less than 15 seconds and more than 15 seconds in different colors.

Then we will plot those two graphs together.

In [23]:
hpG2 = []
hpT2 = []
hpGx2 = []
hpTx2 = []
xx2 = []
xxx2 = []
for i in range(len(hyperParametersAvg2)):
    if hpGaps2[i] < 1 and hpTime2[i] < 15:
        hpG2.append(hpGaps2[i])
        hpT2.append(hpTime2[i])
        xx2.append(x2[i])
    elif hpGaps2[i] < 1 and hpTime2[i] >= 15:
        hpGx2.append(hpGaps2[i])
        hpTx2.append(hpTime2[i])
        xxx2.append(x2[i])
        
hpG3 = []
hpT3 = []
hpGx3 = []
hpTx3 = []
xx3 = []
xxx3 = []
for i in range(len(hyperParametersAvg3)):
    if hpGaps3[i] < 1 and hpTime3[i] < 15:
        hpG3.append(hpGaps3[i])
        hpT3.append(hpTime3[i])
        xx3.append(x3[i])
    elif hpGaps3[i] < 1 and hpTime3[i] >= 15:
        hpGx3.append(hpGaps3[i])
        hpTx3.append(hpTime3[i])
        xxx3.append(x3[i])
        
print("Change2 - Gap% < 1 and Time < 15: " + str(len(hpG2)))
print("Change3 - Gap% < 1 and Time < 15: " + str(len(hpG3)))
print("Change2 - Gap% < 1 and Time > 15: " + str(len(hpGx2)))
print("Change3 - Gap% < 1 and Time > 15: " + str(len(hpGx3)))

        
trace = go.Scatter(
    x = xx2,
    y = hpG2,
    mode = 'markers',
    name = '2-Ch G<1',
    line = dict(color = ('rgb(0, 0, 255)'))
)

trace2 = go.Scatter(
    x = xx2,
    y = hpT2,
    mode = 'markers',
    name = '2-Ch T<15',
    line = dict(color = ('rgb(0, 0, 255)'))
)

trace3 = go.Scatter(
    x = xx3,
    y = hpG3,
    mode = 'markers',
    name = '3-Ch G<1',
    line = dict(color = ('rgb(255, 0, 255)'))
)

trace4 = go.Scatter(
    x = xx3,
    y = hpT3,
    mode = 'markers',
    name = '3-Ch T<15',
    line = dict(color = ('rgb(255, 0, 255)'))
)

trace5 = go.Scatter(
    x = xxx2,
    y = hpGx2,
    mode = 'markers',
    name = '2-Ch G<1',
    line = dict(color = ('rgb(255, 255, 0)'))
)

trace6 = go.Scatter(
    x = xxx2,
    y = hpTx2,
    mode = 'markers',
    name = '2-Ch T>15',
    line = dict(color = ('rgb(255, 255, 0)'))
)

trace7 = go.Scatter(
    x = xxx3,
    y = hpGx3,
    mode = 'markers',
    name = '3-Ch G<1',
    line = dict(color = ('rgb(255, 0, 0)'))
)

trace8 = go.Scatter(
    x = xxx3,
    y = hpTx3,
    mode = 'markers',
    name = '3-Ch T>15',
    line = dict(color = ('rgb(255, 0, 0)'))
)


data = [trace, trace2, trace3, trace4, trace5, trace6, trace7, trace8]

layout = go.Layout(
    title='Gap and Time of Runs',
    xaxis=dict(
        title='Run Number',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Gaps and CPU Times',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Gap and Time of Runs')

Change2 - Gap% < 1 and Time < 15: 2
Change3 - Gap% < 1 and Time < 15: 0
Change2 - Gap% < 1 and Time > 15: 9
Change3 - Gap% < 1 and Time > 15: 3


As a last thing, I sorted the above data with increasing Gap%

In [24]:
hpSort = []
for i in range(len(hpG2)):
    hpSort.append([hpG2[i], hpT2[i], xx2[i], 2])
for i in range(len(hpG3)):
    hpSort.append([hpG3[i], hpT3[i], xx3[i], 3])
for i in range(len(hpGx2)):
    hpSort.append([hpGx2[i], hpTx2[i], xxx2[i], 2])
for i in range(len(hpGx3)):
    hpSort.append([hpGx3[i], hpTx3[i], xxx3[i], 3])

print(len(hpSort))
hpSort.sort()
for i in range(len(hpSort)):
    print(str(i+1) + ". " + str(hpSort[i]))

14
1. [0.43343653250773995, 19.999290628001223, 8, 2]
2. [0.4953560371517028, 33.659829815510335, 24, 2]
3. [0.6811145510835913, 20.40611341497674, 23, 2]
4. [0.7739938080495357, 26.006113652013667, 66, 3]
5. [0.7739938080495357, 52.93161781931703, 79, 2]
6. [0.804953560371517, 41.82187628024539, 51, 2]
7. [0.8978328173374613, 18.85205335065275, 39, 2]
8. [0.8978328173374613, 36.59149251378694, 77, 3]
9. [0.9287925696594427, 15.280272892515494, 57, 3]
10. [0.9287925696594428, 13.815296297920487, 59, 2]
11. [0.9597523219814241, 19.93221562601602, 14, 2]
12. [0.9907120743034055, 14.056041178025271, 6, 2]
13. [0.9907120743034055, 34.33376442418048, 77, 2]
14. [0.9907120743034056, 22.581284446872722, 41, 2]


As we can observe the first 2 parametrization gives very promissing results. Especially the first one gives both a very good gap percentage and a very good timing. So, let's write their parameter settings.

* (8) nSize: 2 - Acceptance: 0.75 - L: 100 - alpha: 0.95 - gama: 1.1
  * Expected Values:
    * Objective Value: 648.80
    * %Gap: 0.43
    * No. of Cycles: 66.40
    * CPU Time(s): 20.00
    
* (24) nSize: 2 - Acceptance: 0.75 - L: 1000 - alpha: 0.9 - gama: 1.15
  * Expected Values:
    * Objective Value: 649.20
    * %Gap: 0.49
    * No. of Cycles: 33.60
    * CPU Time(s): 33.66

### Main Solver

This is the main solver of the code. We will run the algorithm with both of our chosen parameters twice, total of four runs. Then we will take the best solutions among four solution. 

In [81]:
params1 = [2, 0.75, 100, 0.95, 1.1]
params2 = [2, 0.75, 1000, 0.9, 1.15]

# Start Timer
t1 = time.clock()

# Solve twice with first parameters

# Neighborhood search replace size
nSize = params1[0]

# How many times to reheat
nReheat = 2

initialTemp = setInitialTemp(params1[1], nSize)
initialL = params1[2]
alpha = params1[3]
gama = params1[4]
reheated = True
bestSolutions = []
totalIterations = []

SA_GAP(initialTemp, initialL, alpha, gama, nSize, nReheat, bestSolutions, totalIterations)

# Solve twice with second parameters


nSize = params2[0]
nReheat = 2
initialTemp = setInitialTemp(params2[1], nSize)
initialL = params2[2]
alpha = params2[3]
gama = params2[4]
reheated = True
SA_GAP(initialTemp, initialL, alpha, gama, nSize, nReheat, bestSolutions, totalIterations)


# Stop Timer
t2 = time.clock()


Objective Value: 1699 - %Gap: 0.06 - No. of Cycles: 73.00 - CPU Time(s): 104.29
Objective Value: 1698 - %Gap: 0.00 - No. of Cycles: 78.00 - CPU Time(s): 168.52
Objective Value: 1700 - %Gap: 0.12 - No. of Cycles: 36.00 - CPU Time(s): 103.57
Objective Value: 1701 - %Gap: 0.18 - No. of Cycles: 36.00 - CPU Time(s): 99.66


### Results

This is the output part. You can observe the best solution of the previously run code.

In [83]:
bSol = bestSolutions[0]
bSolVal = calculateCost(bestSolutions[0])
for sol in bestSolutions:
    if bSolVal > calculateCost(sol):
        bSolVal = calculateCost(sol)
        bSol = sol

        
solDict = {}
print("Solution:")
for i in range(len(bSol)):
    if bSol[i]+1 not in solDict:
        solDict[bSol[i]+1] = [i+1]
    else:
        solDict[bSol[i]+1].append(i+1)

for i in range(m):
    print(str(i+1) + ": " + str(solDict[i+1]))
print()

print("Objective Value:")
print(bSolVal)
print()

print("%Gap:")
G = 100 * (bSolVal-optimalObjective) / optimalObjective
print("%.2f" %G)
print()

print("No. of Cycles: (Total of " + str(len(bestSolutions)) + " runs)")
nCycle = sum(totalIterations)
print("%.2f" %nCycle)
print()

print("CPU Time (s): (Total of " + str(len(bestSolutions)) + " runs)")
timePassed = (t2-t1)
print("%.2f" %timePassed)

Solution:
1: [6, 13, 17, 18, 21, 23, 29, 41, 47, 49, 51, 68, 73, 79, 81, 96]
2: [4, 26, 27, 34, 36, 46, 50, 60, 65, 69, 72, 76, 84, 88, 93, 94, 98]
3: [10, 14, 16, 25, 28, 30, 32, 33, 35, 37, 55, 59, 62, 67, 70, 71, 75, 78, 85, 90, 92, 100]
4: [1, 3, 5, 7, 9, 12, 19, 22, 24, 31, 42, 48, 52, 53, 56, 57, 58, 61, 63, 80, 82, 83, 87, 95, 99]
5: [2, 8, 11, 15, 20, 38, 39, 40, 43, 44, 45, 54, 64, 66, 74, 77, 86, 89, 91, 97]

Objective Value:
1698

%Gap:
0.00

No. of Cycles: (Total of 4 runs)
223.00

CPU Time (s): (Total of 4 runs)
482.12
